# Generating table for linkers/osmolytes

This notebook describes the steps to generate the

In [ ]:
import pandas as pd

In [ ]:
linkers_dict = {}

## Add the amino acids as linker reactions in the vacuole of both the guard cell and mesophyll cell. They have an osmotic effect in the guard cell but not the mesophyll

In [ ]:
amino_acid_list = [
    '4_AMINO_BUTYRATE',
    'LEU',
    'GLY',
    'VAL',
    'L_ALPHA_ALANINE',
    'ASN',
    'MET',
    'LYS',
    'THR',
    'ILE',
    'SER',
    'GLT',
    'TRP',
    'GLN',
    'TYR',
    'L_ASPARTATE',
    'PRO',
    'PHE',
    'HIS',
    'bHIS',
    'CYS',
    'ARG',
]

for cell, osmolarity in zip(["me", "gc"], [0, 1]):
    for amino_acid in amino_acid_list:
        linkers_dict[f"{amino_acid}_v_{cell}"] = osmolarity

## Use vacuolar linkers from previous diel paper for mesophyll non-osmotic linkers

In [ ]:
me_linkers = [
    "MAL",
    "aMAL",
    "SUCROSE",
    "CIT",
    "aCIT",
]

for me_linker in me_linkers:
    linkers_dict[f"{me_linker}_v_me"] = 0

## Add non-osmotic apoplastic linkers based on what has been found to be present in the apoplast of guard cells

In [ ]:
a_linkers = [
    'Cl',
    'FRU',
    'GLC',
    'K',
    'MAL',
    'NITRATE',
    'SUCROSE',
    'aMAL',
]

for a_linker in a_linkers:
    linkers_dict[f"{a_linker}_a"] = 0

## Add osmotic linkers to cytoplasm and vacuole of GC based on what has been hypothesised to accumulate

In [ ]:
gc_c_linkers = [
    'CIT',
    'Cl',
    'FRU',
    'GLC',
    'K',
    'MAL',
    'NITRATE',
    'SUCROSE',
]

for gc_c_linker in gc_c_linkers:
    linkers_dict[f"{gc_c_linker}_c_gc"] = 1

In [ ]:
gc_v_linkers = [
    'CIT',
    'Cl',
    'FRU',
    'GLC',
    'K',
    'MAL',
    'NITRATE',
    'SUCROSE',
    'aCIT',
    'aMAL',
]

for gc_v_linker in gc_v_linkers:
    linkers_dict[f"{gc_v_linker}_v_gc"] = 1

## Add starch as a non-osmotic linker reaction in both the gc and me plastid

In [ ]:
linkers_dict[f"STARCH_p_gc"] = 0
linkers_dict["STARCH_p_me"] = 0

## Save as .csv file

In [ ]:
osmolytes_df = pd.DataFrame.from_dict(linkers_dict, orient="Index", columns=["Osmotic Coefficient"])
osmolytes_df.index.name = "Linker"
osmolytes_df.to_csv("../inputs/osmolytes.csv")

## Generate a prettier DF for investigating here if wanted

In [ ]:
def get_compartment_cell(linker):

    split_name = linker.split("_")
    cell = split_name[-1]
    if cell != "a":
        compartment = split_name[-2]
        metabolite = split_name[:-2]
    else:
        compartment = "NA"
        metabolite = split_name[:-1]

    if type(metabolite) == list:
        metabolite = "_".join(metabolite)

    return metabolite, compartment, cell

In [ ]:
osmolytes_df_prettyindex = pd.MultiIndex.from_tuples([get_compartment_cell(i) for i in osmolytes_df.index], names=[
                                                     "Metabolite", "Compartment", "Cell"])
osmolytes_df.index = osmolytes_df_prettyindex
osmolytes_df = osmolytes_df.sort_index()
osmolytes_df

Osmotic Coefficient
Metabolite       Compartment Cell                     
4_AMINO_BUTYRATE v           gc                      1
                             me                      0
ARG              v           gc                      1
                             me                      0
ASN              v           gc                      1
...                                                ...
aMAL             NA          a                       0
                 v           gc                      1
                             me                      0
bHIS             v           gc                      1
                             me                      0

[77 rows x 1 columns]

In [ ]:
def return_list_of_items(groupobject):
    return ", ".join(list(groupobject.values.astype("str")))

In [ ]:
linkers_met_dict = {
    "SUCROSE": "Sucrose",
    "GLC": "Glucose",
    "FRU": "Fructose",
    "MAL": "Malate",
    "CIT": "Citrate",
    "Cl": "Cl",
    "K": "K",
    "NITRATE": "Nitrate",
    "aMAL": "aMalate",
    "STARCH": "Starch",
    "aCIT": "aCitrate",
    "bHIS": "bHis",
}

osmolytes_tidy = osmolytes_df.reset_index().set_index(["Osmotic Coefficient", "Cell", "Compartment"]).sort_index()

leftovers = set(osmolytes_tidy.Metabolite) - set(linkers_met_dict.keys())

for l in leftovers:
    linkers_met_dict[l] = l.capitalize()

osmolytes_tidy.Metabolite = osmolytes_tidy.Metabolite.map(lambda x: linkers_met_dict[x])
osmolytes_tidy = osmolytes_tidy.groupby(["Osmotic Coefficient", "Cell", "Compartment"]).agg(return_list_of_items)
osmolytes_tidy.to_csv("../plant_cell_paper/linkers_list_tidy.csv")
osmolytes_tidy

Metabolite
Osmotic Coefficient Cell Compartment                                                   
0                   a    NA           Cl, Fructose, Glucose, K, Malate, Nitrate, Suc...
                    gc   p                                                       Starch
                    me   p                                                       Starch
                         v            4_amino_butyrate, Arg, Asn, Citrate, Cys, Gln,...
1                   gc   c            Citrate, Cl, Fructose, Glucose, K, Malate, Nit...
                         v            4_amino_butyrate, Arg, Asn, Citrate, Cys, Cl, ...